# Amazon Comprehend - Custom Document Classification Example

- Data Source: https://www.kaggle.com/datasets/shivamkushwaha/bbc-full-text-document-classification

In [44]:
# Standard Library
import typing as T
import os
import json
import random
import zipfile
import dataclasses


# Third Party Library
from rich import print as rprint
from pathlib_mate import Path
from s3pathlib import S3Path, context
from boto_session_manager import BotoSesManager, AwsServiceEnum

import polars as pl

bsm = BotoSesManager(profile_name="aws_data_lab_sanhe_us_east_2")
context.attach_boto_session(bsm.boto_ses)

ch_client = bsm.get_client(AwsServiceEnum.Comprehend)

dir_here = Path(os.getcwd())
dir_docs = dir_here / "docs"
path_bbc_zip = dir_here / "bbc.zip"
dir_bbc = dir_here / "bbc"
path_manifest_csv = dir_here / "manifest.csv"

s3dir_root = S3Path.from_s3_uri("s3://aws-data-lab-sanhe-for-everything-us-east-2/poc/2022-11-05-document-classification-example/").to_dir()
s3path_manifest_csv = s3dir_root / "manifest.csv"
s3dir_test_data = s3dir_root.joinpath("test-data").to_dir()
s3dir_predict = s3dir_root.joinpath("predict").to_dir()
print(f"preview s3dir_root: {s3dir_root.console_url}")

preview s3dir_root: https://console.aws.amazon.com/s3/buckets/aws-data-lab-sanhe-for-everything-us-east-2?prefix=poc/2022-11-05-document-classification-example/


In [35]:
# @dataclasses.dataclass
# class Config:
#     s3dir_root: S3Path = dataclasses.field()
#
# config = Config(
#     s3dir_root=S3Path.from_s3_uri("s3://aws-data-lab-sanhe-for-everything/poc/2022-11-05-document-classification-example/").to_dir()
# )

## Unzip Dataset

In [9]:
if not dir_bbc.exists():
    with zipfile.ZipFile(path_bbc_zip.abspath) as f:
        f.extractall(path=dir_here.abspath)

## Split Train and Test


In [25]:
@dataclasses.dataclass
class Document:
    path: Path = dataclasses.field()
    label: str = dataclasses.field()


doc_list: T.List[Document] = list()
for dir in dir_bbc.select_dir(recursive=False):
    for path in dir.select_by_ext(".txt"):
        doc = Document(path=path, label=dir.basename)
        doc_list.append(doc)

n_doc = len(doc_list)
n_train = int(0.7 * n_doc)
n_test = n_doc - n_train

train_indices: T.Set[int] = set(random.sample(list(range(n_doc)), n_train))
train_doc_list: T.List[Document] = list()
test_doc_list: T.List[Document] = list()
for ind, doc in enumerate(doc_list):
    if ind in train_indices:
        train_doc_list.append(doc)
    else:
        test_doc_list.append(doc)

print(f"total docs = {n_doc}")
print(f"train docs = {n_train}")
print(f"test docs = {n_test}")

total docs = 2225
train docs = 1557
test docs = 668


In [36]:
rows: T.List[T.Dict[str, T.Any]] = list()
for doc in train_doc_list:
    content = doc.path.read_text(encoding="utf-8", errors="ignore")
    line_content = content.replace("\n", " ").replace("\r", " ")
    rows.append(dict(
        label=doc.label,
        content=line_content,
    ))
df = pl.DataFrame(rows)
df.write_csv(path_manifest_csv.abspath, has_header=False)
s3path_manifest_csv.upload_file(path_manifest_csv.abspath, overwrite=True)

## Create Document Classifier

In [40]:
classifier_name = "MyClassifier"
classifier_arn = f"arn:aws:comprehend:{bsm.aws_region}:{bsm.aws_account_id}:document-classifier/{classifier_name}"
# create_response = ch_client.create_document_classifier(
#     InputDataConfig=dict(
#         DataFormat="COMPREHEND_CSV",
#         S3Uri=s3path_manifest_csv.uri,
#     ),
#     DataAccessRoleArn="arn:aws:iam::669508176277:role/sanhe-comprehend-admin-access",
#     DocumentClassifierName="MyClassifier",
#     LanguageCode="en",
#     Mode="MULTI_CLASS",
# )
# rprint(create_response)

In [43]:
print("upload test data to s3 ...")
print(f"  preview s3dir_test_data: {s3dir_test_data.console_url}")
for doc in test_doc_list:
    s3path = s3dir_test_data.joinpath(f"{doc.label}-{doc.path.basename}")
    s3path.upload_file(doc.path.abspath)
print("  done")

upload test data to s3 ...
  preview s3dir_test_data: https://console.aws.amazon.com/s3/buckets/aws-data-lab-sanhe-for-everything-us-east-2?prefix=poc/2022-11-05-document-classification-example/test-data/
  done


In [45]:
start_response = ch_client.start_document_classification_job(
    InputDataConfig=dict(
        S3Uri=s3dir_test_data.uri,
        InputFormat="ONE_DOC_PER_FILE",
    ),
    OutputDataConfig=dict(
        S3Uri=s3dir_predict.uri,
    ),
    DataAccessRoleArn="arn:aws:iam::669508176277:role/sanhe-comprehend-admin-access",
    DocumentClassifierArn=classifier_arn,
)
print(start_response)

{'JobId': 'c9899916d3ee54ea34a8bd9bb51ba1ca', 'JobArn': 'arn:aws:comprehend:us-east-2:669508176277:document-classification-job/c9899916d3ee54ea34a8bd9bb51ba1ca', 'JobStatus': 'SUBMITTED', 'ResponseMetadata': {'RequestId': '6e0bc2a5-c28a-4dbc-9a76-610627c0f3f2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6e0bc2a5-c28a-4dbc-9a76-610627c0f3f2', 'content-type': 'application/x-amz-json-1.1', 'content-length': '182', 'date': 'Sat, 05 Nov 2022 19:10:35 GMT'}, 'RetryAttempts': 0}}


In [56]:
lines = dir_here.joinpath("predictions.json").read_text().strip().split("\n")
results = [
    json.loads(line.strip())
    for line in lines
]
n_test = len(results)
n_good = 0
for record in results:
    file = record["File"]
    true_label = file.split("-")[0]
    predict_label = list(sorted(
        record["Classes"],
        key=lambda x: x["Score"],
        reverse=True
    ))[0]["Name"]
    if true_label == predict_label:
        n_good += 1
print(f"accuracy = {n_good / n_test}")

accuracy = 0.9835329341317365
